In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import Dropout
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

/home/renan/.local/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
def accent_remover(x):
    try:
        float(x)
        return x
    except:
        return normalize('NFKD',x).encode('ASCII', 'ignore').decode('ASCII')

In [3]:
# Importar dataset
dataset = pd.read_csv('datasets/datatran2017.csv', sep= ',', encoding='ISO-8859-1').dropna().drop_duplicates()
dataset = dataset.transform([accent_remover])

/home/renan/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1,2,3,4,7,8,9,10,11,12,13,14,15,16,17,28,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# Pega as dados das colunas:
# data_inversa	dia_semana	horario	uf	br	km	condicao_metereologica	latitude	longitude
X = dataset.iloc[:, 11:17].values
y = dataset.iloc[:, 22].values
y=np.array(y)
print(X)
print(y)

[['Com Vitimas Feridas' 'Plena Noite' 'Decrescente' 'Garoa/Chuvisco'
  'Simples' 'Reta']
 ['Com Vitimas Feridas' 'Plena Noite' 'Decrescente' 'Chuva' 'Simples'
  'Nao Informado']
 ['Com Vitimas Fatais' 'Plena Noite' 'Decrescente' 'Ceu Claro' 'Simples'
  'Curva']
 ...
 ['Com Vitimas Fatais' 'Anoitecer' 'Decrescente' 'Nublado' 'Simples'
  'Curva']
 ['Com Vitimas Feridas' 'Plena Noite' 'Crescente' 'Nublado' 'Simples'
  'Reta']
 ['Com Vitimas Feridas' 'Anoitecer' 'Crescente' 'Nublado' 'Dupla' 'Reta']]
[16  8  7 ... 30  8 13]


In [5]:
# Converte os dados de string para number gerando as categorias
encoder = LabelEncoder()
X[:,0] = encoder.fit_transform(X[:,0])
X[:,1] = encoder.fit_transform(X[:,1])
X[:,2] = encoder.fit_transform(X[:,2])
X[:,3] = encoder.fit_transform(X[:,3])
X[:,4] = encoder.fit_transform(X[:,4])
X[:,5] = encoder.fit_transform(X[:,5])
print(X)

ct=ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 1, 2, 3])], remainder='passthrough')
X=np.array(ct.fit_transform(X))
print(X)

[[1 2 1 2 2 5]
 [1 2 1 1 2 3]
 [0 2 1 0 2 0]
 ...
 [0 1 1 6 2 0]
 [1 2 0 6 2 5]
 [1 1 0 6 0 5]]
[[0.0 1.0 0.0 ... 0.0 2 5]
 [0.0 1.0 0.0 ... 0.0 2 3]
 [1.0 0.0 0.0 ... 0.0 2 0]
 ...
 [1.0 0.0 0.0 ... 0.0 2 0]
 [0.0 1.0 0.0 ... 0.0 2 5]
 [0.0 1.0 0.0 ... 0.0 0 5]]


In [6]:
# Divide o dataset em conjunto de treino e conjunto de teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
# Converte os dados para uma mesma faixa de valores de 0 a 1
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
np.shape(X_train)

(35825, 20)

In [8]:
# Inicializa a ANN
regressor = Sequential()

# Adiciona a primeira camada
regressor.add(Dense(units=6, activation='relu'))
#regressor.add(Dropout(0.2))

# Segunda camada
regressor.add(Dense(units=6, activation='relu'))
#regressor.add(Dropout(0.2))

# Camada de saida
regressor.add(Dense(units=1, activation='sigmoid'))
#regressor.add(Dropout(0.2))

# Quarta camada
#regressor.add(GRU(units = 100))
#regressor.add(Dropout(0.2))

# Camada de saida
#regressor.add(Dense(units = 9))
2
# Compila
regressor.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

# Executa o treinamento
regressor.fit(X_train, y_train, epochs = 25, batch_size = 32)


Epoch 1/25
1120/1120 [==============================] - 1s 543us/step - loss: -1629.2527 - accuracy: 0.0000e+00
Epoch 2/25
1120/1120 [==============================] - 1s 524us/step - loss: -23747.8867 - accuracy: 0.0000e+00
Epoch 3/25
1120/1120 [==============================] - 1s 558us/step - loss: -90252.0547 - accuracy: 0.0000e+00
Epoch 4/25
1120/1120 [==============================] - 1s 592us/step - loss: -211913.2500 - accuracy: 0.0000e+00
Epoch 5/25
1120/1120 [==============================] - 1s 478us/step - loss: -395198.8438 - accuracy: 0.0000e+00
Epoch 6/25
1120/1120 [==============================] - 1s 481us/step - loss: -647259.1875 - accuracy: 0.0000e+00
Epoch 7/25
1120/1120 [==============================] - 1s 646us/step - loss: -975298.5000 - accuracy: 0.0000e+00
Epoch 8/25
1120/1120 [==============================] - 1s 578us/step - loss: -1385874.3750 - accuracy: 0.0000e+00
Epoch 9/25
1120/1120 [==============================] - 1s 598us/step - loss: -1887700.7500

In [9]:
# Executa predição
y_pred=regressor.predict(X_test)
y_pred

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [10]:
y_pred=(y_pred>0.5)
y_pred

array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [12]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
print(cm)

[[   0    0    0 ...    0    0    0]
 [ 240    0    0 ...    0    0    0]
 [1449    0    0 ...    0    0    0]
 ...
 [   1    0    0 ...    0    0    0]
 [   1    0    0 ...    0    0    0]
 [   1    0    0 ...    0    0    0]]
